# Data Preprocessing

In [50]:
import pandas as pd
import re
import os
import tqdm

from PIL import Image
import cv2

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Sampler

## Structured data

1. clean dataset
2. extract the classes for each video
3. extract dominant action for each video

In [12]:
train_data = pd.read_csv("../data/Charades_v1_train.csv")
train_data

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33
2,0IH69,6RE8,Bedroom,6.0,5.0,Yes,A person is seen leaving a cabinet. They then ...,book;box;cabinet;shelf,A person is standing in a bedroom. They walk o...,NaN,30.25
3,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33
4,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38
...,...,...,...,...,...,...,...,...,...,...,...
7980,7K2CS,HJZQ,Garage,6.0,6.0,Yes,Person enters the garage while sneezing. Perso...,chair;clothes;door;food;sandwich;shirt;spoon,"A enters through a doorway, sneezes, then clos...",c065 17.60 31.00;c067 17.60 31.00;c153 0.00 5....,30.08
7981,S2A89,KL48,Bathroom,7.0,7.0,Yes,"A person takes a chair and walks it over, plac...",chair;door,A PERSON IS TAKING A CHAIR FROM ONE ROOM TO TH...,c006 4.00 10.80;c141 4.40 10.90;c151 12.80 20....,19.29
7982,01O27,18IT,Bathroom,6.0,7.0,Yes,A person enters a bathroom and closes the door...,door;floor;mirror,A person is walking towards the bathroom. A pe...,c006 5.10 11.50;c008 0.50 6.60;c124 39.00 47.0...,46.08
7983,2MJ72,6RE8,Bedroom,6.0,6.0,Yes,A person opens a window in their laundry room....,door;towel;window,A person opens a window and looks out of it. ...,c006 11.00 17.00;c037 20.70 31.00;c092 0.60 8....,30.25


In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7985 entries, 0 to 7984
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            7985 non-null   object 
 1   subject       7985 non-null   object 
 2   scene         7985 non-null   object 
 3   quality       7968 non-null   float64
 4   relevance     7930 non-null   float64
 5   verified      7985 non-null   object 
 6   script        7985 non-null   object 
 7   objects       7982 non-null   object 
 8   descriptions  7985 non-null   object 
 9   actions       7811 non-null   object 
 10  length        7985 non-null   float64
dtypes: float64(3), object(8)
memory usage: 686.3+ KB


Remove all the rows that doesn't have a specified action

In [70]:
cleaned_data = train_data.dropna(subset = ['actions'])
cleaned_data.reset_index(drop = True, inplace = True)

In [71]:
cleaned_data.describe()

,quality,relevance,length
count,7795.000000,7758.000000,7811.000000
mean,5.585247,6.227636,29.885159
std,1.186276,1.150637,9.406070
min,1.000000,1.000000,2.330000
25%,5.000000,6.000000,26.960000
50%,6.000000,7.000000,30.540000
75%,6.000000,7.000000,32.120000
max,7.000000,7.000000,194.330000


In [72]:
def extract_class(s):
    class_labels = re.findall('c\d+', s)
    return class_labels

In [73]:
extracted_classes = cleaned_data['actions'].apply(extract_class)
num_classes = extracted_classes.apply(len)

cleaned_data.loc[:, 'classes'] = extracted_classes
cleaned_data.loc[:, 'num_classes'] = num_classes

cleaned_data

/home/kayne/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length,classes,num_classes
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83,"[c092, c147]",2
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33,"[c098, c075, c127, c153]",4
2,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33,"[c018, c141, c148, c006, c002, c150, c000]",7
3,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38,"[c015, c107]",2
4,S6MPZ,EA2K,Home Office / Study (A room in a house used fo...,6.0,6.0,Yes,A person is eating at the desk and lying the p...,chair;desk;food;phone;plate;snacks;table,A person in a blue shirt is eating snacks off ...,c009 0.00 4.30;c011 0.00 39.00;c015 0.00 39.00...,38.46,"[c009, c011, c015, c019, c156, c059, c061, c06...",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7806,7K2CS,HJZQ,Garage,6.0,6.0,Yes,Person enters the garage while sneezing. Perso...,chair;clothes;door;food;sandwich;shirt;spoon,"A enters through a doorway, sneezes, then clos...",c065 17.60 31.00;c067 17.60 31.00;c153 0.00 5....,30.08,"[c065, c067, c153, c155, c097, c141, c156]",7
7807,S2A89,KL48,Bathroom,7.0,7.0,Yes,"A person takes a chair and walks it over, plac...",chair;door,A PERSON IS TAKING A CHAIR FROM ONE ROOM TO TH...,c006 4.00 10.80;c141 4.40 10.90;c151 12.80 20....,19.29,"[c006, c141, c151, c097]",4
7808,01O27,18IT,Bathroom,6.0,7.0,Yes,A person enters a bathroom and closes the door...,door;floor;mirror,A person is walking towards the bathroom. A pe...,c006 5.10 11.50;c008 0.50 6.60;c124 39.00 47.0...,46.08,"[c006, c008, c124, c096]",4
7809,2MJ72,6RE8,Bedroom,6.0,6.0,Yes,A person opens a window in their laundry room....,door;towel;window,A person opens a window and looks out of it. ...,c006 11.00 17.00;c037 20.70 31.00;c092 0.60 8....,30.25,"[c006, c037, c092, c033]",4


Some videos have more than 1 action/class, for the baseline model, we will just take the action that spans a longer duration.

In [74]:
def get_dominant_action(actions, classes):
    actions = actions.split(";")
    
    longest_duration = 0
    longest_action_idx = 0
    for i, action in enumerate(actions):
        start, end = set(map(float, action.split(" ")[1:]))
        duration = end - start
        if duration > longest_duration:
            longest_duration = duration
            longest_action_idx = i

    return classes[longest_action_idx]

In [75]:
cleaned_data['dominant_action'] = cleaned_data.apply(lambda x: get_dominant_action(x.actions, x.classes), axis = 1)
cleaned_data

/tmp/ipykernel_141272/4051372197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['dominant_action'] = cleaned_data.apply(lambda x: get_dominant_action(x.actions, x.classes), axis = 1)


,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length,classes,num_classes,dominant_action
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83,"[c092, c147]",2,c147
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33,"[c098, c075, c127, c153]",4,c127
2,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33,"[c018, c141, c148, c006, c002, c150, c000]",7,c150
3,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38,"[c015, c107]",2,c015
4,S6MPZ,EA2K,Home Office / Study (A room in a house used fo...,6.0,6.0,Yes,A person is eating at the desk and lying the p...,chair;desk;food;phone;plate;snacks;table,A person in a blue shirt is eating snacks off ...,c009 0.00 4.30;c011 0.00 39.00;c015 0.00 39.00...,38.46,"[c009, c011, c015, c019, c156, c059, c061, c06...",10,c011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7806,7K2CS,HJZQ,Garage,6.0,6.0,Yes,Person enters the garage while sneezing. Perso...,chair;clothes;door;food;sandwich;shirt;spoon,"A enters through a doorway, sneezes, then clos...",c065 17.60 31.00;c067 17.60 31.00;c153 0.00 5....,30.08,"[c065, c067, c153, c155, c097, c141, c156]",7,c065
7807,S2A89,KL48,Bathroom,7.0,7.0,Yes,"A person takes a chair and walks it over, plac...",chair;door,A PERSON IS TAKING A CHAIR FROM ONE ROOM TO TH...,c006 4.00 10.80;c141 4.40 10.90;c151 12.80 20....,19.29,"[c006, c141, c151, c097]",4,c151
7808,01O27,18IT,Bathroom,6.0,7.0,Yes,A person enters a bathroom and closes the door...,door;floor;mirror,A person is walking towards the bathroom. A pe...,c006 5.10 11.50;c008 0.50 6.60;c124 39.00 47.0...,46.08,"[c006, c008, c124, c096]",4,c008
7809,2MJ72,6RE8,Bedroom,6.0,6.0,Yes,A person opens a window in their laundry room....,door;towel;window,A person opens a window and looks out of it. ...,c006 11.00 17.00;c037 20.70 31.00;c092 0.60 8....,30.25,"[c006, c037, c092, c033]",4,c037


## Video processing

1. Choose sequence length for each video
2. sample frames from each video
    - Loop the video if it is too short

Create Dataset to use batching

In [94]:
class VideoDataset(Dataset):
    def __init__(self, vid_names, vid_dir, seq_len=100):
        super(VideoDataset, self).__init__()
        self.vid_names = vid_names
        self.vid_dir = vid_dir
        self.seq_len = seq_len
        self.transform = self.get_transforms()
    
    def get_transforms(self):
        "for MobileNetv2" 
        return transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        
    def __getitem__(self, idx):
        '''
        adapted from bleedai
        '''
        
        video_path = os.path.join(self.vid_dir, f"{self.vid_names[idx]}.mp4")
        video_reader = cv2.VideoCapture(video_path)

        frames_list = []
        # Get the total number of frames in the video.
        video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

        # Calculate the the interval after which frames will be added to the list.
        skip_frames_window = max(int(video_frames_count / self.seq_len), 1)

        for frame_counter in range(self.seq_len):
            # Set the current frame position of the video.
            frame_position = frame_counter * skip_frames_window % video_frames_count
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_position)
            success, frame = video_reader.read() 

            if not success:
                break

            processed_frame = self.transform(Image.fromarray(frame))
            frames_list.append(processed_frame)

        video_reader.release()
        return frames_list
    
    def __len__(self):
        return len(self.vid_names)

### testing

In [95]:
vid_names = cleaned_data.loc[0:5, 'id']
vid_dataset = VideoDataset(vid_names, os.path.join("..", "data", "Charades_v1"))

In [96]:
dataloader = DataLoader(vid_dataset, batch_size = 1, shuffle = True, num_workers=3)

In [98]:
x = next(iter(dataloader))

In [103]:
len(x)

100

In [104]:
x[0].size()

torch.Size([1, 3, 224, 224])

---